In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service

# Setup Selenium WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True) # this keeps the window open post execution/errors/ changing frames (explained later)

prefs = {"download.default_directory": '/Users/justinpaul/Downloads',  # note need to change this
         "directory_upgrade": True}
chrome_options.add_experimental_option("prefs", prefs)

#creates a chrome webdriver 
driver = webdriver.Chrome(service=service, options=chrome_options)
actions = ActionChains(driver)

url = 'https://www.mden.com/'

# Navigate to the URL
driver.get(url)
time.sleep(3) 

# Extracting product names and prices
products = driver.find_elements(By.CLASS_NAME, 'product')
product_data = []
for product in products:
    try:
        name = product.find_element(By.CLASS_NAME, 'product-name').text
        price = product.find_element(By.CLASS_NAME, 'price').text
        product_data.append({'Name': name, 'Price': price})
    except:
        continue

# Close the driver
driver.quit()

# Convert to DataFrame
df = pd.DataFrame(product_data)
print(df.head())

# Save the data
df.to_csv('mden_products.csv', index=False)


In [ ]:
def categorize_product(name):
    name = name.lower()
    if 'hat' in name or 'cap' in name:
        return 'Hat'
    elif 'jeans' in name or 'pants' in name or 'trousers' in name:
        return 'Pants'
    elif 'shirt' in name or 't-shirt' in name or 'jacket' in name:
        return 'Shirt'
    else:
        return 'Other'  # Default category

# Apply the function to the DataFrame
df['Category'] = df['Name'].apply(categorize_product)

In [ ]:
import pandas as pd

# Convert list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Convert price from string to numeric
df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)

# Save the cleaned data
df.to_csv('cleaned_data.csv', index=False)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# Split data into category like hats and price
X = df.drop('Category', axis=1)
y = df['Price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Initialize and train a model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions and evaluate the model
predictions = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))
